# 15.8 文本生成：GPT与ChatGPT
- **目录**
  - 15.8.1 GPT与ChatGPT概述
  - 15.8.2 GPT模型架构
  - 15.8.3 ChatGPT训练技术和过程
    - 15.8.3.1 人类反馈强化学习
    - 15.8.3.2 TAMER框架
    - 15.8.3.3 ChatGPT的训练过程
    - 15.8.3.4 GPT-4的训练技术
  - 15.8.4 GPT实现
    - 15.8.4.1 数据准备
    - 15.8.3.2 构建模型
    - 15.8.4.3 模型训练
    - 15.8.4.4 文本生成

## 15.8.1 GPT与ChatGPT概述

- GPT是 OpenAI 的一系列预训练模型，GPT的全称是 **Generative Pre-Trained Transformer**。
  - 顾名思义，GPT 的目标是通过 Transformer，使用预训练技术得到通用的语言模型。
- 目前已经公布论文的有 GPT-1、GPT-2、GPT-3。
- 最新的GPT-4有技术报告，但是技术细节公布的不多。
- GPT-4 Turbo是GPT-4 的升级版本，发布于 2023 年 11 月 6 日，主要特点如下：
  - 更大的上下文窗口：GPT-4 Turbo 具有 128K 的上下文长度，能够处理更长的文本内容，提升了对长文本的理解和处理能力。
  - 更新的知识库：其知识库已更新至 2023 年 4 月，能够提供更及时和准确的信息。
  - 多模态支持：支持文生图模型 DALL·E 3、具有视觉输入能力的 GPT-4 Turbo 以及新的声音合成模型（TTS）等多模态 API。
  - 可定制微调：允许开发人员创建 ChatGPT 自定义版本，进行特定领域的预训练和强化学习后训练。
- GPT-4o 是 OpenAI 发布的一款多模态大模型，于 2024 年 5 月 14 日发布。
  - 其中“o”代表“omni”，该词意为“全能”，源自拉丁语“omnis”。
  - GPT-4o 模型可以使 ChatGPT 能够处理 50 种不同的语言，并可以接受文本、音频和图像三者组合作为输入，并生成文本、音频和图像的任意组合输出。
  - 可以在 232 毫秒内对音频输入做出反应，与人类在对话中的反应时间相近。
  - 性能方面，在传统基准测试中，GPT-4o 在文本、推理和编码等方面实现了与 GPT-4 Turbo 级别相当的性能，同时在多语言、音频和视觉功能方面的表现分数也超过了之前的模型。
  - 相较于 2023 年 11 月推出的 GPT-4 Turbo，GPT-4o 在处理速度上提升达到 200%，同时在价格上也下降了 50%，并分阶段集成至 OpenAI 的各个产品之中。
- 在2025年2月OpenAI发布GPT-4.5。
- 2025年1月国内DeepSeek AI助手上线正式提供服务，引起巨大轰动。

- ChatGPT是构建在GPT之上的系列模型，早期ChatGPT基于GPT-3.5进行微调而成，ChatGPT Plus则是基于GPT-4。
- OpenAI团队在GPT-3.5 基础上，使用**人类反馈强化学习（Reinforcement Learning from Human Feedback, RLHF）** 训练模型。
  - 首先使用了人类标注师撰写约1.2w-1.5w条问答数据，并用其作为基础数据预训练。
  - 随后让预训练好的**监督微调模型（Supervised Fine-Tuning, SFT）** 针对新问题列表生成若干条回答，并让人类标注师对这些回答进行排序。
  - 这些回答的排名内容将以配对比较的方式生成一个新的**奖励模型（Reward Mode，RM）**。
  - 最后让奖励模型在更大的数据集上重新训练SFT，并将最后两个步骤反复迭代以获得最终的模型。

## 15.8.2 GPT模型架构

- GPT其实并不是一种新型架构，其结构类似于transformer模型中的解码器，并在庞大的数据集上进行了训练。
- 原始模型如下图所示：
<center><img src='../img/15_8_1.png' width=800px></center>
<center>图15.8.1 GPT模型图</center>
- 注：上图来源于[此网址](https://zhuanlan.zhihu.com/p/604625917)

- 具体讲，transformer模型的Decoder部分包含MHA（多头注意力）和MMHA（掩码多头自注意力），而GPT只保留MMHA，去掉MMA。
- 这确保了 GPT 只能关注上文的信息，从而达到单向模型的目的。
- 如下图所示：
<center><img src = '../img/15_8_3.png' width=400px></center>
<center>图15.8.2 GPT模型与Encoder对比</center>

- GPT-1通过**自左向右生成式**的构建预训练任务，然后得到一个通用的预训练模型，这个模型和BERT一样都可用来做下游任务的微调。
  - GPT-1当时在9个NLP任务上取得了SOTA的效果，但GPT-1使用的模型规模和数据量都比较小，这也就促使了GPT-2的诞生。
- 对比GPT-1，GPT-2并未在模型结构做大规模修改，只是使用了更多参数的模型和更多的训练数据。
  - GPT-2最重要的思想是提出了“所有的有监督学习都是无监督语言模型的一个子集”的思想，这个思想也是**提示学习（Prompt Learning）** 的前身。
  - GPT-2在诞生之初也引发了不少的轰动，它生成的新闻足以欺骗大多数人类，达到以假乱真的效果。
- GPT-3被提出时，除了它远超GPT-2的效果外，更令人瞩目的是其1750亿参数量。
  - GPT-3除了能完成常见的NLP任务外，研究者意外的发现GPT-3在写SQL，JavaScript等语言的代码，进行简单的数学运算也有不俗表现。
  - GPT-3的训练使用了**情境学习（In-context Learning）**，它是一种**元学习（Meta-learning）** 。
  - 元学习的核心思想在于通过少量的数据寻找一个合适的初始化范围，使得模型能够在有限的数据集上快速拟合，并获得不错的效果。

-----------
- **说明：元学习（Meta-learning）和情境学习（In-context Learning）？**
  - 元学习（Meta-learning）和情境学习（In-context Learning）是两种复杂的学习概念，特别是在人工智能（AI）和机器学习（ML）领域中，这些理念被广泛探索和应用。
  - 元学习（Meta-learning）：又被称为“学习的学习”，是指让机器学习模型学会如何更有效地学习的过程。这种学习方式的目的在于使模型能够通过较少的数据、较快的速度或更高的效率来学习新任务，而不是在每次面对新任务时都从头开始学习。元学习尝试找到模型学习任务的一般策略，以便当面对新的、未见过的任务时，能够快速适应。
  - 元学习的关键在于找到有效的学习算法（学习策略），这可以通过多种方式实现，包括但不限于：
    - **模型无关的元学习（Model-Agnostic Meta-Learning, MAML）**：这种方法旨在通过对一系列不同任务的学习，找到一个好的模型初始化，这个初始化使模型可以通过少量梯度更新步骤和少量样本就快速适应新任务。
    - - **优化方法**：通过设计学习过程中的优化算法，来增强模型适应新任务的能力。这可能包括修改反向传播算法使其更适合新任务的快速学习。
      - - **记忆方法**：利用外部记忆机制或增强内部表示来提升模型对以往任务的记忆能力，从而使模型能够在面对新任务时，利用过往的知识。
  - 情境学习（In-context Learning）：是一种使模型能够根据提供在其输入中的信息（情境）来调整其行为的能力，特别是指无需显式重新训练或微调的情况下。
    - 例如，最新一代的大型语言模型（如GPT-3等）就表现出了强大的情境学习能力：它们可以通过阅读一个问题的描述和相关的例子，然后直接在该情境中生成对应的答案或完成指定的任务。
  - 情境学习的一个关键特点是模型的多功能性和灵活性，它能够理解并应对各种不同类型的请求，而不需要为每种请求单独训练一个专用模型。这种能力基于以下两点：
    - **大量的训练数据**：模型在训练过程中看到了大量的语言结构和信息，因此能够理解和处理各式各样的输入。
    - **强大的内部表示**：模型能够学会如何将输入的信息转化为内部表示，这些内部表示捕捉到了输入数据的关键特征和语义，使得模型可以在这些表示的基础上进行推理和生成答案。
------------

- 从GPT-1, GPT-2到GPT-3的结构演进如下：
  - GPT-1:
    - 12层transformer，每层12个注意力头。
  - GPT-2的改进：
    - GPT-2有48层，使用1600维向量进行词嵌入。
    - 将层归一化移动到每个子块的输入，并在最终的自注意块后增加一层归一化。
    - 修改初始化的残差层权重，缩放为原来的$1/\sqrt N$，$N$是残差层的数量。
    - 特征向量维数从768扩展到1600，词表扩大到50257。    
  - GPT-3的改进：
    - GPT-3有96层，每层有96个注意力头。
    - GPT-3的单词嵌入大小从GPT-2的1600增加到12888。
    - 上下文窗口大小从GPT-2的1024增加到GPT-3的2048。
    - 采用**交替密度**和**局部带状稀疏注意力模式**。
   

- GPT-1,GPT-2(xl),GPT-3参数对比表如下：


| 参数                   | GPT-1              | GPT-2 (xl)         | GPT-3            |
|------------------------|--------------------|--------------------|--------------------|
| 参数量（Parameters）   | 117M               | 1.5B               | 175B               |
| 层数（Layers）         | 12                 | 48                 | 96                 |
| 注意力头数（Attention Heads） | 12             | 25                 | 96                 |
| 嵌入维度（Embedding Dim）  | 768                | 1600               | 12288              |
| 最大序列长度（Max Seq Length） | 512          | 1024               | 2048               |
| 数据集大小 （Dataset Size）            | 5GB          | 40GB               | 45TB(处理前) 400B(处理后)             |

- 参数解释
  - **参数量（Parameters）**：模型中的可训练参数总数。GPT-3 的参数量是 GPT-1 的近 1500 倍。
  - **层数（Layers）**：transformer 网络中的层数。GPT-3 的层数是 GPT-1 的 8 倍，捕捉更深层次的语言特征。
  - **注意力头数（Attention Heads）**：每个transformer 层中的自注意力头的数量。GPT-3 的注意头数是 GPT-1 的 8 倍，显著提高了上下文理解能力。
  - **嵌入维度（Embedding Dim）**：输入和注意力机制的向量维度。GPT-3 的嵌入维度远大于 GPT-1，允许其表示更复杂的信息。
  - **最大序列长度（Max Seq Length）**：模型能够处理的最大输入序列长度。GPT-3 的最大序列长度是 GPT-1 的 4 倍，可以处理更长的文本输入。


## 15.8.3 ChatGPT训练技术和过程
从GPT-3.5开始，OpenAI加入了两种技术：**人类反馈强化学习**和**TAMER**框架。

### 15.8.3.1 人类反馈强化学习
- 人类反馈强化学习（Reinforcement Learning from Human Feedback, RLHF）通过将人类的评价和反馈融入到智能体的学习过程中，能够有效提高其在复杂和动态环境中的表现。
- RLHF是一种结合人类反馈来强化机器学习模型的方法，它特别适用于训练在复杂、模糊或动态环境中表现优秀的智能体。
- RLHF的核心思想是通过人类提供的指示、奖励或评分来逐步改进智能体的策略，以达到更符合人类预期或更优的行为表现。
- 这种方法不仅能让智能体更快速地学习到高效的策略，还能借助人类智慧和经验来避开潜在的危险和误区，从而在许多实际应用中展现出广泛的前景和巨大的优势。
- RLHF是GPT-3.5这个版本被引入的。
  - 与GPT-3的主要区别在于，GPT-3.5新加入了被称为**人类反馈强化学习**的技术。
  - 这一训练范式增强了人类对模型输出结果的调节，并且对结果进行了更具理解性的排序。
- 在InstructGPT中，以下是“goodness of sentences”的评价标准。
  - 真实性：是虚假信息还是误导性信息？
  - 无害性：它是否对人或环境造成身体或精神上的伤害？
  - 有用性：它是否解决了用户的任务？
- RLHF核心概念和工作流程如下：
  - **智能体（Agent）**: 执行操作并观察结果的主体。
  - **环境（Environment）**: 智能体与之交互的外部世界，包含智能体的行动空间和状态空间。
  - **行动（Action）**: 智能体在每个时间步选择的一种行为。
  - **状态（State）**: 描述当前环境的各种信息和智能体的情境。
  - **奖励（Reward）**: 从环境或人类反馈中得到的数值，用于指导智能体的学习过程。
  - **策略（Policy）**: 智能体在特定状态下选择行动的规则或方法。
- 应用RLHF的过程通常包括以下步骤：
  - **智能体执行动作**：智能体在环境中通过一定的策略来选择和执行动作。
  - **观察和反馈**：智能体观察执行动作后的结果。人类观察者对当前的状态和动作组合进行评价，给出反馈（如奖励或惩罚）。
  - **更新策略**：智能体根据收到的反馈，调整其策略以在未来类似的情况下做出更优决策。
  - **重复循环**：上述步骤重复进行，智能体通过不断试错和人类反馈不断改进。

### 15.8.3.2 TAMER框架
- **TAMER（Training an Agent Manually via Evaluative Reinforcement，评估式强化人工训练代理）** 框架，将人类标记者引入到Agents的学习循环中，可以通过人类向Agents提供奖励反馈（即指导Agents进行训练），从而快速达到训练任务目标。
- TAMER可以将人类标记者的知识，以奖励信反馈的形式训练Agent，加快其快速收敛。
- TAMER不需要标记者具有专业知识或编程技术，语料成本更低。通过TAMER+RL（Reinforcement Learning, 强化学习），借助人类标记者的反馈，能够增强从马尔可夫决策过程(Markov Decision Process, MDP) 奖励进行强化学习的过程。
- TAMER架构在强化学习中的应用:
  - 具体实现上，人类标记者扮演对话的用户和人工智能助手，提供对话样本，让模型生成一些回复，然后标记者会对回复选项打分排名，将更好的结果反馈回模型中。
  - Agents同时从两种反馈模式中学习——人类强化和马尔可夫决策过程奖励作为一个整合的系统，通过奖励策略对模型进行微调并持续迭代。
  - 在此基础上，ChatGPT 可以比 GPT-3 更好的理解和完成人类语言或指令，模仿人类，提供连贯的有逻辑的文本信息的能力。

### 15.8.3.3 ChatGPT的训练过程
以GPT-3.5为例，ChatGPT的训练过程分为以下三个阶段：
- 第一阶段：训练监督策略模型。
  - GPT-3.5本身很难理解人类不同类型指令中蕴含的不同意图，也很难判断生成内容是否是高质量的结果。
  - 为了让GPT-3.5初步具备理解指令的意图，首先会在数据集中随机抽取问题，由人类标注人员，给出高质量答案。
  - 然后用这些人工标注好的数据来微调 GPT-3.5模型获得**SFT（Supervised Fine-Tuning）** 模型。
  - 此时的SFT模型在遵循指令/对话方面已经优于 GPT-3，但不一定符合人类偏好。
- 第二阶段：训练**奖励模型（Reward Mode，RM）**。
  - 这个阶段的主要是通过人工标注训练数据（约33K个数据），来训练回报模型。
  - 在数据集中随机抽取问题，使用第一阶段生成的模型，对于每个问题，生成多个不同的回答。
  - 人类标注者对这些结果综合考虑给出排名顺序。这一过程类似于教练或老师辅导。
  - 接下来，使用这个排序结果数据来训练奖励模型。
  - 对多个排序结果，两两组合，形成多个训练数据对。
  - RM模型接受一个输入，给出评价回答质量的分数。
  - 这样，对于一对训练数据，调节参数使得高质量回答的打分比低质量的打分要高。
- 第三阶段：采用**PPO（Proximal Policy Optimization，近端策略优化）** 强化学习来优化策略。
  - PPO的核心思路在于将Policy Gradient中On-policy的训练过程转化为Off-policy，即将**在线学习** 转化为**离线学习**，这个转化过程被称之为Importance Sampling。
  - 这一阶段利用第二阶段训练好的奖励模型，靠奖励打分来更新预训练模型参数。
  - 在数据集中随机抽取问题，使用PPO模型生成回答，并用上一阶段训练好的RM模型给出质量分数。
  - 把回报分数依次传递，由此产生策略梯度，通过强化学习的方式以更新PPO模型参数。
  - 最后不断重复第二和第三阶段，通过迭代，会训练出更高质量的ChatGPT模型。

### 15.8.3.4 GPT-4的训练技术
**GPT-4**模型及其系统产品**ChatGPT Plus**的参数规模据说已达1.8万亿。根据有关网上[技术文章](https://zhuanlan.zhihu.com/p/626463196)的研究，GPT-4技术方案可能采用了如下策略和算法：
- **zero-shot**、**one-shot**和**few-shot**的学习能力：这个提升的理论依据很大可能是因为大模型的**涌现能力（emergent ability）**。
- 逻辑推理能力：用到了大模型的**思维链（Chain of Thought，CoT）** 以及**自提升能力（Self-Improve Ability）** 。
- 理解图像能力：推测借鉴了OpenAI著名的多模态模型**CLIP（对比语言-图像预处理，Contrastive Language–Image Pre-Training）** 或者是微软的多模态模型**KOSMOS-1**。
- 更安全的文本生成能力：这一部分技术报告中介绍的比较多，主要是专家测试，幻觉检测以及**RBRM（基于规则的奖励模型，Rule-based Reward Model）**。
- 更强的编程能力：推测这一部分借鉴了OpenAI的著名的代码生成模型：**CodeX**。
- 处理其它语言的能力：推测可能借鉴了XLM等跨语言预训练模型的思想，或是因为涌现能力强化了GPT-4在其它语种上的表现效果。
- 处理更长序列的能力：推测这一部分用到了处理长输入的模型**Transformer-XL**或者OpenAI提出的可以降低长数据复杂度的**Sparse Transformer**。

----------

- **说明：**
- **（1）何为对比语言-图像预训练CLIP？**
  - **对比语言-图像预训练（Contrastive Language–Image Pre-Training，CLIP）** 是一种深度学习方法，用于同时理解图像内容和相关的文本信息。它通过对比学习的框架来优化模型，使得模型能够更好地将图像和对应的文本描述联系起来。
  - CLIP能够大幅提高模型处理和理解图文信息任务的能力。
  - 通过这种联合预训练方法，模型不仅能学习到丰富的视觉特征，还能学习到复杂的语义信息，从而在多种跨模态任务上实现更优的性能。
  - 在对比语言-图像预训练中，主要目标是训练一个能够理解图像及其相关文本的表示的模型。
  - 这种方法通常涉及到两个主要的组件：
    - 一是视觉编码器，用于提取图像特征；
    - 二是语言编码器，用于提取文本特征。
    - 这两个编码器被同时训练，以确保它们能够生成相似的表示形式。
    - 当输入的文本描述与图像内容相关时，模型会试图将二者的表示拉近；相反，如果输入的文本描述与图像不相关，模型则会推开二者的表示。
  - CLIP技术细节如下：
    - **对比损失函数**：这项技术的核心在于使用对比损失（Contrastive Loss），也称为三元组损失（Triplet Loss），来训练模型。这种损失函数鼓励模型使得相匹配的图像和文本对的表示更接近，同时使得不匹配的图像和文本对的表示相互远离。
    - **多模态学习**：Contrastive Language–Image Pre-Training是一项多模态学习方法，因为它涉及到处理并理解两种或两种以上的模态（图像和文本）。这种多模态学习方法能够显著提高模型在多种任务上的表现，比如图像标注、视觉问题回答（Visual Question Answering, VQA）以及跨模态信息检索等。
    - **预训练和微调**：这种技术通常包含两个阶段。首先，在预训练阶段，模型在大规模的图文配对数据集上进行训练，目的是学习通用的视觉-语言表示。然后，在微调阶段，模型在特定任务的较小数据集上进行进一步训练，以优化其在特定任务上的表现。



- **（2）何为涌现能力（emergent ability）？**
  - 大语言模型的**涌现能力（emergent ability）** 是指当语言模型达到一定的规模和复杂性时，它们能够表现出原本在训练过程中未明确训练或设计的新能力和行为。
  - 这些能力可能包括对新问题的理解、综合信息的能力，甚至一些创造性的任务执行，这些都是在训练数据中没有直接指导的。
  - 以下是一些涌现能力的例子和详细说明：
    - **更强的语境理解**：随着模型的规模增大，它们开始更准确地理解复杂的语境关系和更微妙的语言使用。这意味着大型模型能够根据上下文提供更精确的回应。
    - **知识内化和推理**：大型语言模型在其庞大的数据库中积累了大量知识，并且随着规模扩展，模型能够更好地内化这些信息，并在必要时进行逻辑推理。
    - **自我修正能力**：在某些情况下，大型模型会表现出能够从自身的错误中学习并进行自我修正的能力，即使在训练中没有特定地教给它们这样做
    - **多步骤任务处理**：当语言模型的规模让它们能够处理更复杂的多步骤任务时，比如先进行研究再回答问题，这种能力没有被明确地教给模型，而是随着训练数据和参数规模的增长而自然出现的。
    - **创造性生成**：对于如OpenAI的DALL·E这样的模型，随着规模的增长，它们开始表现出能够创造新图像的能力，这些图像不仅仅是对训练数据的复制，而是原创的、有创意的产物。
    - **自然语言理解的深度**：大型模型能够理解和使用双关语、隐喻、幽默或其他复杂的语言表达方式。



- **（3）何为"Zero-shot"、"One-shot"和"Few-shot"？**
  - "Zero-shot"、"One-shot"和"Few-shot"学习是指深度学习模型在不同数量的示例下进行学习的能力。
  - **Zero-shot learning**：在此场景中，模型能够在没有任何具体例子即示例的情况下理解和执行新任务。模型利用已有的知识和理解来推断任务要求并试图给出正确的输出。这通常依赖于模型的泛化能力。
    - 大语言模型如GPT可以进行zero-shot学习。如果你要求它回答一个问题，比如“谁是第一位踏上月球的人？”即使模型没有被明确地训练来回答这个特定的问题，它可能已经在背景材料或相关文本中学会了答案，因此可以给出正确答案，即尼尔·阿姆斯特朗。
  - **One-shot learning**：在one-shot学习中，模型会看到一个示例，然后就需要执行与该示例相关的任务。模型根据单个实例理解任务要求，并应用于新的情况。
     - 在使用GPT模型时，你可以给它一个示例，比如展示一个格式化的日期：“March 14, 1879 - Albert Einstein's birthdate”。然后询问，“April 15, 1452”，期望它识别这是描述日期和著名人物生日的方式，模型将从给出的一个样本中学习并尝试返回：“April 15, 1452 - Leonardo da Vinci's birthdate”。
  - **Few-shot learning**：在few-shot学习场景下，模型会看到少量的示例来理解新任务。模型利用这些有限的情境来调整自己对任务的理解，并在新的情况中使用这个概念。
    - 如果你问GPT一个分类问题，并给它几个示例分类，如：“苹果 - 水果，胡萝卜 - 蔬菜”，然后提出一个新项，“番茄”，模型将根据先前的示例推断番茄是个蔬菜（尽管生物学上是水果，但在烹饪中通常被视为蔬菜）。
  - 在这些场景中，模型的预训练部分学习了大范围的语言模式和知识，这使得在没有专门针对新任务进行额外数据训练的情况下，模型还是能够处理这些任务。
  - OpenAI的GPT模型特别擅长这些学习方法，因为它们在大规模数据集上进行了训练，从而理解了大量的概念和任务。这样的模型可以应用于各种不同的情境，只需很少或没有额外的示例来展示如何完成新任务。

- **（4）何为RBRM（基于规则的奖励模型，Rule-based Reward Model）？**
  - GPT使用的RBRM（基于规则的奖励模型，Rule-based Reward Model）技术是一种用于改善语言模型输出质量和安全性的方法。RBRM通过一组预定义的规则来评估模型的输出，并据此提供正向或负向的反馈。这些规则通常是由人类专家制定的，旨在引导模型生成更符合期望的响应。
  - 以下是RBRM的一些关键特点和应用场景：
    - **规则定义**：RBRM的核心是一组规则，这些规则定义了模型输出的期望属性，比如内容的准确性、适当性或安全性。这些规则可以是具体的指令，也可以是评价标准。
    - **零样本分类器**：RBRM使用零样本（Zero-shot）分类器，这意味着它们不需要针对特定任务进行训练。这些分类器能够根据预定义的规则对行为或事件进行分类。
    - **奖励信号**：在强化学习（Reinforcement Learning, RL）框架中，RBRM提供了额外的奖励信号，指导模型学习并优化其行为，使其更加符合既定的安全和质量标准。
    - **多输入处理**：RBRM可以接受多种输入，包括提示（可选）、策略模型的输出，以及人类编写的评估准则。这些输入帮助模型理解任务要求，并据此生成响应。
    - **分类输出**：RBRM将根据提供的规则集对模型的输出进行分类。例如，它可以指示模型将响应分类为期望的拒绝、不期望的拒绝、包含不允许的内容，或是安全且非拒绝的响应。
    - **微调过程**：RBRM通常用于微调（Fine-tuning）阶段，在此阶段，模型通过与RBRM的交互学习如何更好地遵循规则并生成合适的输出。
    - **安全性和质量控制**：RBRM有助于确保模型的输出不包含不当内容，如仇恨言论、歧视性语言或不准确的信息，从而提高模型的安全性和输出质量。
    - **迭代改进**：通过RBRM的反馈，模型开发者可以识别和解决模型的潜在问题，不断迭代和改进模型的性能。



- **（4）何为Transformer-XL？**
  - **Transformer-XL**是一种基于Transformer架构的预训练语言模型，它旨在解决传统Transformer模型在处理长序列数据时的长度限制问题。
  - 在标准的Transformer模型中，由于其自注意力机制的限制，通常只能处理固定长度的序列，这被称为“长度限制”问题。
  - Transformer-XL通过引入一种新颖的“可重复的缓存机制”来克服这一限制。
  - Transformer-XL的关键特点包括：
    - **可重复的缓存机制**：Transformer-XL引入了段级别的循环机制，允许模型在处理新的输入段时，重复使用之前段的隐藏状态。这种机制使得模型能够维持长距离依赖关系，并且能够处理比标准Transformer更长的序列。
    - **前向缓存和后向缓存**：Transformer-XL使用前向缓存（将当前段的输出传递到下一个段）和后向缓存（将之前段的信息传递回当前段），这有助于在不同段之间传递信息，增强模型对长距离依赖的捕捉能力。
    - **缓解长度限制**：通过这种方式，Transformer-XL能够处理比自身实际长度限制更长的序列，因为它可以利用之前处理的序列信息。
    - **改善长距离依赖问题**：Transformer-XL特别适合于需要理解长距离依赖的语言任务，如文档摘要、文本生成等
    - **预训练任务**：Transformer-XL通常在大量文本数据上进行预训练，以学习语言的通用表示，然后可以在特定任务上进行微调。
    - **性能提升**：在多项自然语言处理任务上，Transformer-XL已经展现出优于标准Transformer和其他序列模型的性能。
  

- **（5）何为近端策略优化（Proximal Policy Optimization, PPO）**
  - **近端策略优化**是一种用于**强化学习**的**策略优化算法**，由OpenAI提出。
    - 该技术在现实应用中表现出色，被广泛用于训练复杂任务中的智能体。
    - **PPO**在训练稳定性和样本效率方面进行了优化，是深度强化学习领域中最受欢迎和有效的算法之一。
  - **核心思想**:
    - PPO的主要目标是优化策略，使智能体在不断试错中学到最优的行为策略。
    - 其核心思想是通过对策略进行小范围的更新，从而避免大幅度策略变化可能导致的不稳定性。
  - PPO的**主要特点**：
    - **基于策略的优化**：
      - PPO是一种策略梯度方法，直接优化智能体的策略。
      - 与值函数方法不同，策略方法直接优化策略而不需要通过值函数间接优化。
    - **限制策略变化幅度**：
      - PPO通过限制每次更新的策略变化，防止策略更新过度，保持训练过程的稳定性。
    - **分阶段更新**：
      - PPO在每个训练阶段中多次更新策略，而不是在每一步更新。
      - 在一个学习周期结束后，利用所有的经验进行优化。
  - **PPO的工作流程**：PPO的核心步骤包括采样、计算优势函数、构建目标函数和策略更新。
    - **采样**：智能体在环境中执行一系列动作，收集经验样本，这些样本包括**状态、动作、奖励和下一状态**等信息。
    - **计算优势函数（Advantage Function）**：优势函数用于衡量某个动作在某状态下相比其他动作的优越性。计算方式通常为：
      $$A_t = R_t + \gamma V(s_{t+1}) - V(s_t)$$
      - $ R_t $ 是在时间步 $ t $ 获得的奖励。
      - $ V(s_{t+1}) $ 和 $ V(s_t) $ 分别是状态 $ s_{t+1} $ 和 $ s_t $ 的值函数估计。
      - $\gamma$ 是折扣因子。
    - **构建目标函数**: PPO的目标函数通过剪切损失函数的方式限制策略变化幅度，具体形式为：
    $$L^{\text{CLIP}}(\theta) = \mathbb{E}_t \bigg[ \min\left( r_t(\theta) \hat{A}_t, \text{clip}(r_t(\theta), 1 - \epsilon, 1 + \epsilon) \hat{A}_t \right) \bigg] $$
      - $ r_t(\theta) = \frac{\pi_\theta(a_t|s_t)}{\pi_{\theta_{\text{old}}}(a_t|s_t)} $ 是新的策略和旧的策略比率。
      - $\hat{A}_t$ 是优势函数估计值。
      - $\epsilon$ 是一个超参数，用于控制策略变化范围。
      - $\text{clip}(r, 1 - \epsilon, 1 + \epsilon)$ 用于限制目标函数内部比率的变化幅度。
    - **策略更新**: 通过梯度下降法优化目标函数，以更新策略参数 $\theta$。
  - 优势和应用:
    - **稳定性高**：通过限制每次更新的策略变化范围，PPO在训练过程中更加稳定，避免了策略剧烈波动带来的不稳定性。
    - **样本效率高**：PPO在每个学习阶段多次使用采样数据，提高了样本利用效率。
    - **实现简便**：PPO相较于一些复杂的策略优化算法，更易于实现且计算效率高。

---------------

## 15.8.4 GPT实现

- 本节使用GPT-2的实现为例进行说明，代码借鉴Andrej karpathy的[nanoGPT](https://github.com/karpathy/nanoGPT)。

### 15.8.4.1 数据准备

- 为了最快了解GPT的原理以及实战技巧，karpathy提供了一个训练GPT莎士比亚作品的数据集。
- 首先，我们将其下载为单个(1MB)文件，并将其从原始文本转换为一个大的整数流。
- 训练参数：
  - 以字符为单位的数据集长度:1115394
  - 所有唯一性字符，即词表中的词元:
    - $ !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz$
  - vocab size: 65
  - train有1003854个词元
  - val有111540个词元

In [4]:
"""
准备字符级的文本数据
"""
import os
import pickle
import requests
import numpy as np

# 下载莎士比亚微型数据集，并文本字符串读入到data变量
input_file_path = '../data/gpt2/tinyshakespeare_input.txt'
with open(input_file_path, 'r') as f:
    data = f.read()
print(f"length of dataset in characters: {len(data):,}")


# 获取文本中的所有字符，包括大小写字母和标点符号
# 作为词表的词元，是字符级词元
chars = sorted(list(set(data)))
vocab_size = len(chars)
print("all the unique characters:", ''.join(chars))
print(f"vocab size: {vocab_size:,}")

# 创建字符和整数之间的映射集
stoi = { ch:i for i,ch in enumerate(chars) } # 字符到整数
itos = { i:ch for i,ch in enumerate(chars) } #整数到字符
def encode(s):
    return [stoi[c] for c in s] # 编码器: 输入一个字符串, 输出一个整数列表
def decode(l):
    return ''.join([itos[i] for i in l]) # 解码器: 输入一个整数列表, 输出一个字符串

# 将数据分割成训练集和测试集
n = len(data)
train_data = data[:int(n*0.9)]
val_data = data[int(n*0.9):]

# 将训练和测试验证集编码成整数
train_ids = encode(train_data)
val_ids = encode(val_data)
print(f"train has {len(train_ids):,} tokens")
print(f"val has {len(val_ids):,} tokens")

# 将训练和测试集的ID输出到二进制文件，
# 通俗讲就是将文本数据集编码成词表对应的整数后，输出到二进制文件
train_ids = np.array(train_ids, dtype=np.uint16)
val_ids = np.array(val_ids, dtype=np.uint16)
train_ids.tofile(os.path.join('../data/gpt2', 'train.bin'))
val_ids.tofile(os.path.join('../data/gpt2', 'val.bin'))

# 保存元信息，以帮助稍后进行的编码/解码操作
meta = {
    'vocab_size': vocab_size,
    'itos': itos,
    'stoi': stoi,
}
with open(os.path.join('../data/gpt2', 'meta.pkl'), 'wb') as f:
    pickle.dump(meta, f)

length of dataset in characters: 1,115,394
all the unique characters: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
vocab size: 65
train has 1,003,854 tokens
val has 111,540 tokens


In [2]:
# 字符与整数之间的映射
#stoi,itos

### 15.8.3.2 构建模型

- 构建GPT2模型
- 模型的组件包括
  - 层归一化
  - 因果自注意力
  - 多层感知机

- **层归一化**
  - 该代码块定义了一个名为`LayerNorm`的类，它继承自`torch.nn.Module`，是一个PyTorch模块，用来实现层归一化（Layer Normalization）功能。
  - **层归一化操作**：   层归一化是对神经网络层的输入进行归一化处理，使其具有0的均值和单位方差。
    - 这种归一化是在特征维度上进行的，而不像批归一化是在批量数据维度上。归一化可以帮助减少内部协变量的移动，从而使模型训练更加稳定。
  - **归一化参数**：
    - `self.weight`：一个可学习的参数，提供了对归一化后数据进行缩放的能力。初始化为全1，表示在训练开始时不对归一化后的值进行缩放。
    - `self.bias`：一个可学习的参数，提供了对归一化后数据进行位移的能力。如果`bias`参数为真，则初始化为全0，表示在训练开始时不对归一化后的值进行偏移。
  - **前向传播（`forward`方法）**：   在前向传播当中，对输入数据`input`应用层归一化。
    - 方法`F.layer_norm`是一个调用PyTorch函数库中的层归一化函数。
    - 参数包括输入数据、归一化时要考虑的形状（维度）、缩放权重、偏移偏置以及归一化时考虑的数值稳定性而添加的微小常数（epsilon）1e-5。

In [5]:
import math
import inspect
from dataclasses import dataclass
import torch
import torch.nn as nn
from contextlib import nullcontext
from torch.nn import functional as F
import numpy as np

class LayerNorm(nn.Module):
    '''
    层归一化。
    与OpenAI使用TensorFlow开发的官方模型不同，
    Pytorch模型不带bias=False选项
    '''

    def __init__(self, ndim, bias):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(ndim))
        self.bias = nn.Parameter(torch.zeros(ndim)) if bias else None

    def forward(self, input):
        return F.layer_norm(input, self.weight.shape, self.weight, self.bias, 1e-5)

- **因果自注意力模型**

In [6]:
class CausalSelfAttention(nn.Module):
    '''
    自注意力模型的实现
    '''
    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0 # 确保嵌入维度能被头的数量整除
        # 自注意力所有头的key,query和value映射，以批量为单位
        # 使用一个线性层将输入嵌入映射到3倍的嵌入维度，本例：768->3*768
        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd, bias=config.bias)
        # 输出映射，再次使用一个线性层来映射自注意力操作的输出到原始的嵌入维度
        # 768->768
        self.c_proj = nn.Linear(config.n_embd, config.n_embd, bias=config.bias)
        # 模型正则化，设置Dropout层,dropout=0.2,即保留20%的单元
        self.attn_dropout = nn.Dropout(config.dropout) # 应用于注意力权重上
        self.resid_dropout = nn.Dropout(config.dropout) # 应用于自注意力层输出
        # 注意力头数，嵌入的维度，Dropout的比例参数
        self.n_head = config.n_head # 12
        self.n_embd = config.n_embd # 768
        self.dropout = config.dropout # 0.2
        # flash attention：优化内存使用和计算速度
        self.flash = hasattr(torch.nn.functional, 'scaled_dot_product_attention')
        if not self.flash:
            print("WARNING: using slow attention. Flash Attention requires PyTorch >= 2.0")
            # 因果掩码，以确保注意力只应用于输入序列中的左侧
            self.register_buffer("bias", torch.tril(torch.ones(config.block_size, config.block_size))
                                        .view(1, 1, config.block_size, config.block_size))

    def forward(self, x):
        B, T, C = x.size() # 批量大小, 序列长度, 嵌入维度 (n_embd)

        # 计算批处理中所有自注意力头的查询、键、值，并将自注意力头的维度和序列长度的维度互换
        q, k, v  = self.c_attn(x).split(self.n_embd, dim=2)
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)

        # 自注意力: (B, nh, T, hs) x (B, nh, hs, T) -> (B, nh, T, T)
        if self.flash:
            # 使用 Flash Attention CUDA核，实现更有效的自注意力机制
            y = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=self.dropout if self.training else 0, is_causal=True)
        else:
            # 手工实现自注意力
            att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1))) # 缩放自注意力
            att = att.masked_fill(self.bias[:,:,:T,:T] == 0, float('-inf')) # 掩码自注意力
            att = F.softmax(att, dim=-1)
            att = self.attn_dropout(att)
            y = att @ v # (B, nh, T, T) x (B, nh, T, hs) -> (B, nh, T, hs)
            
        # 注意contiguous的用法，tensor多次transpose会使其丧失连续型，通过contiguous可以将其恢复连续型     
        y = y.transpose(1, 2).contiguous().view(B, T, C) # 重组所有的自注意力输出，恢复原来形状和维度
           
        # 输出映射，同样也需要使用dropout
        y = self.resid_dropout(self.c_proj(y))
        return y

- MLP类作为一个前馈网络层的实现。
  - 在Transformer架构中，这样的前馈网络常见于每一个注意力模块之后的位置，并对序列中的每个位置都执行相同的操作。
  - 这是一个完全连接的网络层，通常用于**特征的非线性变换**。

In [7]:
class MLP(nn.Module):
    '''
    多层感知机作为前馈网络层存在。
    '''
    def __init__(self, config):
        super().__init__()
        self.c_fc    = nn.Linear(config.n_embd, 4 * config.n_embd, bias=config.bias)
        self.gelu    = nn.GELU() # Gaussian Error Linear Unit激活函数
        self.c_proj  = nn.Linear(4 * config.n_embd, config.n_embd, bias=config.bias)
        self.dropout = nn.Dropout(config.dropout)

    def forward(self, x):
        x = self.c_fc(x)
        x = self.gelu(x)
        x = self.c_proj(x)
        x = self.dropout(x)
        return x

- GPTConfig是一个辅助类，用于在命令行端运行时，接受输入的参数值，并覆盖各参数的默认值。
  - 本节对代码进行修改，使之可在notebook中直接运行。

In [8]:
@dataclass
class GPTConfig:
    block_size: int = 1024
    vocab_size: int = 65 # GPT-2 vocab_size of 50257, padded up to nearest multiple of 64 for efficiency
    n_layer: int = 12
    n_head: int = 12
    n_embd: int = 768
    dropout: float = 0.0
    bias: bool = True # True: bias in Linears and LayerNorms, like GPT-2. False: a bit better and faster

- 构建Decoder块，正如前文所述，和transformer原始解码器的结构有所区别。

In [9]:
# GPT解码器块，和transformer的原始解码器有所区别
class Block(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.ln_1 = LayerNorm(config.n_embd, bias=config.bias)
        self.attn = CausalSelfAttention(config)
        self.ln_2 = LayerNorm(config.n_embd, bias=config.bias)
        self.mlp = MLP(config)

    def forward(self, x):
        x = x + self.attn(self.ln_1(x))
        x = x + self.mlp(self.ln_2(x))
        return x

- GPT-2
  - 包括模型的构建、前向传播、参数初始化、优化器配置以及如何用模型进行文本生成。

In [10]:
# GPT-2模型
class GPT(nn.Module):
    def __init__(self, config):
        super().__init__()
        assert config.vocab_size is not None
        assert config.block_size is not None # 决定模型捕捉词元依赖关系的距离
        self.config = config # 使用GPTConfig对象配置模型参数，一般来自命令行输入
        
        # 构造解码器层
        self.transformer = nn.ModuleDict(dict(
            wte = nn.Embedding(config.vocab_size, config.n_embd), # 词嵌入
            wpe = nn.Embedding(config.block_size, config.n_embd), # 位置嵌入
            drop = nn.Dropout(config.dropout),
            h = nn.ModuleList([Block(config) for _ in range(config.n_layer)]),# 6层解码器
            ln_f = LayerNorm(config.n_embd, bias=config.bias)
        ))
        # 最后一个线性层 ，用于将解码器层的输出投影回词表空间，以便计算最终的词元概率分布。
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)           
        self.transformer.wte.weight = self.lm_head.weight

        # 初始化所有参数
        self.apply(self._init_weights)
        # 对残差投影应用特殊缩放初始化
        for pn, p in self.named_parameters():
            if pn.endswith('c_proj.weight'):
                torch.nn.init.normal_(p, mean=0.0, std=0.02/math.sqrt(2 * config.n_layer))

        # 报告参数个数
        print("number of parameters: %.2fM" % (self.get_num_params()/1e6,))

    def get_num_params(self, non_embedding=True):
        '''
        返回模型中参数的个数。
        （1）默认情况下（即在不计算嵌入层参数的情况下），位置嵌入（position embeddings）的参数数量会被从总数中减去。
             位置嵌入用于为序列模型中每个元素提供其位置信息。
             在计算总参数数量时，可能需要把位置嵌入的参数数量排除在外。
        （1）通常情况下，如果进行非嵌入层的参数计数，也会从总数中减去词嵌入（token embeddings）的参数，
             就像位置嵌入一样。
             然而，由于参数共享（parameter sharing）的原因，这些词嵌入参数实际上在模型的最后一层被用作权重。
             所以，尽管词嵌入是嵌入的一部分，通常还是将其参数计入参数总数。
        '''
        n_params = sum(p.numel() for p in self.parameters())
        if non_embedding:
            n_params -= self.transformer.wpe.weight.numel()
        return n_params

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
    # 前向传播
    def forward(self, idx, targets=None):
        device = idx.device
        b, t = idx.size()
        assert t <= self.config.block_size, f"Cannot forward sequence of length {t},  block size is only {self.config.block_size}"
        pos = torch.arange(0, t, dtype=torch.long, device=device) # shape (t)

        # 前向传播模型
        tok_emb = self.transformer.wte(idx) # 词嵌入的形状 (b, t, n_embd)
        pos_emb = self.transformer.wpe(pos) # 位置嵌入的形状 (t, n_embd)
        x = self.transformer.drop(tok_emb + pos_emb) # 两个嵌入相加
        for block in self.transformer.h:
            x = block(x)
        x = self.transformer.ln_f(x)

        # 如果提供了 targets，则计算交叉熵损失；否则，如果是生成任务，则不需要计算损失。
        # 分别运行在训练阶段和生成阶段
        if targets is not None:
            # 根据给定的期望目标，计算损失
            logits = self.lm_head(x)
            # 交叉熵损失
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1), ignore_index=-1)
        else:
            # 推断时间的微型优化(mini-optimization):只转发最后一个位置的lm_head
            logits = self.lm_head(x[:, [-1], :]) # 注意: 使用list [-1] 保存序列维度的大小，即序列的长度
            loss = None

        return logits, loss   

    def configure_optimizers(self, weight_decay, learning_rate, betas, device_type):
        # 从所有候选参数开始
        param_dict = {pn: p for pn, p in self.named_parameters()}
        # 过滤掉不需要的梯度
        param_dict = {pn: p for pn, p in param_dict.items() if p.requires_grad}
        '''
        （1）decay_params: 如果参数的维度大于等于2（通常指的是矩阵，如权重矩阵），
             则认为这些参数需要应用权重衰减。权重衰减主要用于正则化，可以减少模型的过拟合。
        （2）nodecay_params: 如果参数的维度小于2（通常是向量，如偏置项和层规范化的参数），
             这些参数不应用权重衰减。
        （3）len(decay_params)：表示“被权重衰减的参数组中参数矩阵的数量”。
             换句话说，它统计的是有多少个权重矩阵被包含在应用权重衰减的参数列表中。
             例如，如果你有两个卷积层和一个全连接层，这可能会是三——每层一个权重矩阵。
        （4）num_decay_params: 这表示“在所有应用权重衰减的参数矩阵中的总参数数量”。
             这是统计具体有多少个浮点数（权重）将要被优化。计算方法是将每个参数矩阵的元素数量(numel)加起来。
             如果有一个有1000个元素的权重矩阵和一个有500个元素的权重矩阵，则 num_decay_params将会是1500。
        '''
        decay_params = [p for n, p in param_dict.items() if p.dim() >= 2]
        nodecay_params = [p for n, p in param_dict.items() if p.dim() < 2]
        optim_groups = [
            {'params': decay_params, 'weight_decay': weight_decay},
            {'params': nodecay_params, 'weight_decay': 0.0}
        ]
        num_decay_params = sum(p.numel() for p in decay_params)
        num_nodecay_params = sum(p.numel() for p in nodecay_params)
        print(f"num decayed parameter tensors: {len(decay_params)}, with {num_decay_params:,} parameters")
        print(f"num non-decayed parameter tensors: {len(nodecay_params)}, with {num_nodecay_params:,} parameters")

        '''
        创建AdamW优化器，决定是否使用fused操作。
        如果AdamW的fused参数可用，并且device_type是'cuda'，则use_fused将被设置为True。
        fused版本的AdamW将执行梯度下降算法中的几个步骤合并成一个GPU操作：
        参数的更新（包括momentum和variance的计算、权重衰减和参数更新）是作为单个内核运行，而非若干个离散步骤。
        这能够减少GPU内核之间的上下文切换以及减少对全局内存的读写次数。
        '''
        fused_available = 'fused' in inspect.signature(torch.optim.AdamW).parameters
        use_fused = fused_available and device_type == 'cuda'
        extra_args = dict(fused=True) if use_fused else dict()
        optimizer = torch.optim.AdamW(optim_groups, lr=learning_rate, betas=betas, **extra_args)
        print(f"using fused AdamW: {use_fused}")
        return optimizer

    def estimate_mfu(self, fwdbwd_per_iter, dt):
        """ 以A100 bfloat16峰值flops为单位估计模型的flops利用率(MFU) """
        # 首先估计每次迭代的flops次数        
        N = self.get_num_params()
        cfg = self.config
        L, H, Q, T = cfg.n_layer, cfg.n_head, cfg.n_embd//cfg.n_head, cfg.block_size
        flops_per_token = 6*N + 12*L*H*Q*T
        flops_per_fwdbwd = flops_per_token * T
        flops_per_iter = flops_per_fwdbwd * fwdbwd_per_iter
        # 将我们的flops吞吐量表示为A100 bfloat16峰值flops的比率
        flops_achieved = flops_per_iter * (1.0/dt) # per second
        flops_promised = 312e12 # A100 GPU bfloat16峰值flops为312 TFLOPS
        mfu = flops_achieved / flops_promised
        return mfu
        
    @torch.no_grad() # 停止梯度传播
    def generate(self, idx, max_new_tokens, temperature=1.0, top_k=None):
        """
        取索引idx(形状为(b,t)的LongTensor)的条件序列，并完成序列max_new_tokens，每次将预测反馈回模型。
        最好确保在model.eval()模式下执行此操作。
        """
        for _ in range(max_new_tokens):
            # 如果序列上下文长得太长，此处必须裁剪为block_size长度
            idx_cond = idx if idx.size(1) <= self.config.block_size else idx[:, -self.config.block_size:]
            # 前向传播模型以获得序列中索引的logit，模型预测下一个token的每个可能值的原始分数
            logits, _ = self(idx_cond)
            '''
            logits是神经网络最后一层输出的原始分数，是一个三维张量，其形状通常为 (batch_size, 
            sequence_length, vocab_size)：
              - batch_size表示一批处理的数据大小。
              - sequence_length表示序列的长度，或者说是已经生成的token的数量。
              - vocab_size是模型词汇表的大小，即模型可以选择的下一个token的所有可能值。
            当进行文本生成时，每次都基于当前的序列（context）来预测下一个token。
            因此对于每个序列，都只关心其最后一个位置的logits，也就是最新生成的token的分数，
            因为这个位置的分数将用来预测下一个token，这就是logits[:, -1, :]的取法原因。
            '''
            logits = logits[:, -1, :] / temperature # 
            # 将logits裁剪为仅前k个选项，此步是可选的
            if top_k is not None:
                v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
                logits[logits < v[:, [-1]]] = -float('Inf')
            # 应用softmax将对数转换为(规范化)概率
            probs = F.softmax(logits, dim=-1)
            '''
             根据概率分布进行随机采样的一个函数。它的作用是从给定的概率分布中随机抽取样本。
             这个函数在文本生成任务中特别有用，因为它可以根据模型输出的概率分布来选择下一个词。
             基于提供的概率分布（第一个参数张量的每一行），随机地从这些分布中抽取num_samples个样本。
             这对于基于预测概率执行随机决策的场景非常有用，如文本生成中的下一个词的选择，
             因为它允许模型引入随机性，避免每次都选择概率最大的词，而是给予较小概率的词一定的机会，
             这有助于生成更多样化和有趣的文本。
            '''
            idx_next = torch.multinomial(probs, num_samples=1)
            # 将抽样索引附加到运行序列并继续
            idx = torch.cat((idx, idx_next), dim=1)

        return idx

In [7]:
# inspect.signature函数的用法
inspect.signature(torch.optim.AdamW).parameters

mappingproxy({'params': <Parameter "params: Union[Iterable[torch.Tensor], Iterable[Dict[str, Any]]]">,
              'lr': <Parameter "lr: Union[float, torch.Tensor] = 0.001">,
              'betas': <Parameter "betas: Tuple[float, float] = (0.9, 0.999)">,
              'eps': <Parameter "eps: float = 1e-08">,
              'weight_decay': <Parameter "weight_decay: float = 0.01">,
              'amsgrad': <Parameter "amsgrad: bool = False">,
              'maximize': <Parameter "maximize: bool = False">,
              'foreach': <Parameter "foreach: Optional[bool] = None">,
              'capturable': <Parameter "capturable: bool = False">,
              'differentiable': <Parameter "differentiable: bool = False">,
              'fused': <Parameter "fused: Optional[bool] = None">})

### 15.8.4.3 模型训练

- 本示例主要是演示GPT-2模型结构，因此使用karpathy提供的莎士比亚数据进行训练。
- 此微型模型是一个字符级，主要为了便于在PC机上进行训练。

In [8]:
import os
import pickle
import time
import numpy as np
# 最低硬件条件训练的一个微型GPT-2模型
out_dir = '../data/gpt2'
eval_interval = 50 # 评估间隔
eval_iters = 200
log_interval = 10 # 日志间隔

# 在小数据集上过拟合，然后便于在验证集上训练有改善时才保存
always_save_checkpoint = False

bias = False
dataset = '../data'
gradient_accumulation_steps = 1
batch_size = 64
block_size = 256 # 最多256个字符的上下文

# 微型GPT-2模型的参数
n_layer = 6 # 解码器的层数
n_head = 6 # 自注意力的头数
n_embd = 384 # 嵌入的维数
dropout = 0.2 # 暂退的比率

learning_rate = 1e-3 # 对于微型模型，学习率设置比较高
# max_iters = 5000 # 该参数原值，但是在PC机上运行较为缓慢
max_iters = 400
# lr_decay_iters = 5000 
lr_decay_iters = 400 # 通常和max_iters参数值相同
min_lr = 1e-4 # 通常等于learning_rate / 10
init_from = 'scratch'
warmup_iters = 100 # 并非很必要

# device = 'cpu'  # 在CPU上运行
compile = False # do not torch compile the model

iter_num = 0
best_val_loss = 1e9

device='cuda'
dtype = 'float16'
beta1 = 0.9
beta2 = 0.95
weight_decay = 1e-1
device_type = 'cuda' if 'cuda' in device else 'cpu' # for later use in torch.autocast
data_dir = os.path.join(dataset,'gpt2')

In [9]:
config_str = {'out_dir':'../data/gpt2',
'eval_interval' : 250,
'eval_iters' : 200,
'log_interval' : 10,
'always_save_checkpoint' : False,
'bias' :False,
'dataset' : '../data',
'gradient_accumulation_steps' : 1,
'batch_size' : 64,
'block_size' : 256,
'n_layer' : 6,
'n_head' : 6,
'n_embd' : 384,
'dropout' : 0.2,
'learning_rate' : 1e-3,
'max_iters' : 400, 
'lr_decay_iters' : 400,
'min_lr' : 1e-4 ,
'init_from' : 'scratch',
'warmup_iters' : 100,
'compile':False,
'iter_num':0,
'best_val_loss':1e9,
'device':'cuda',
'dtype' : 'float16',
'beta1' : 0.9,
'beta2' : 0.95,
'weight_decay' : 1e-1,
'device_type' : 'cuda' ,
'data_dir' :'../data/gpt2'}

In [10]:
# 获取批量
def get_batch(split):
    if split == 'train':
        '''
        函数使用 numpy的 memmap功能载入对应的二进制数据文件。
        memmap创建了一个内存映射数组，它允许大文件被部分读入内存，
        对于处理不能完全装入RAM的大型数据集特别有用。
        '''
        data = np.memmap(os.path.join(data_dir, 'train.bin'), dtype=np.uint16, mode='r')
    else:
        data = np.memmap(os.path.join(data_dir, 'val.bin'), dtype=np.uint16, mode='r')
        
    # 生成0到len(data) - block_size之间的随机数，size是batch_size，即批量的大小
    ix = torch.randint(len(data) - block_size, (batch_size,))
    # 生成输入序列和标签的批次，本质上是对应token在词表中的索引
    x = torch.stack([torch.from_numpy((data[i:i+block_size]).astype(np.int64)) for i in ix])
    # 标签是输入序列中每个token右边紧邻的一个token，这也是自回归的基础
    y = torch.stack([torch.from_numpy((data[i+1:i+1+block_size]).astype(np.int64)) for i in ix])
    if device_type == 'cuda':
        # 使用 pin_memory()方法准备输入序列的Tensor，以便将之异步地传输到GPU
        x, y = x.pin_memory().to(device, non_blocking=True), y.pin_memory().to(device, non_blocking=True)
    else:
        x, y = x.to(device), y.to(device)
    return x, y

In [11]:
'''
感觉这些参数在训练模型时没用到。
'''
meta_path = '../data/gpt2/meta.pkl'
meta_vocab_size = None
if os.path.exists(meta_path):
    with open(meta_path, 'rb') as f:
        meta = pickle.load(f)
        #print('meta:----------',meta)
    meta_vocab_size = meta['vocab_size']
    print(f"found vocab_size = {meta_vocab_size} (inside {meta_path})")
config_str['meta_path'] = meta_path
config_str['meta_vocab_size'] = meta_vocab_size

found vocab_size = 65 (inside ../data/gpt2/meta.pkl)


In [12]:
# 模型初始化
model_args = dict(n_layer=n_layer, n_head=n_head, n_embd=n_embd, block_size=block_size,
                  bias=bias, vocab_size=None, dropout=dropout) # 设置模型参数
print("Initializing a new model from scratch")
model_args['vocab_size'] = meta_vocab_size if meta_vocab_size is not None else 50304
gptconf = GPTConfig(**model_args)
#print(gptconf) 
model = GPT(gptconf)

Initializing a new model from scratch
number of parameters: 10.65M


In [13]:
model.to(device) # 将模型转移到到GPU
# 初始化GradScaler对象，如果enabled设为False，则梯度伸缩器则不被操作
scaler = torch.cuda.amp.GradScaler(enabled=(dtype == 'float16'))
# 如果在GPU上运行，则在float32和float16两个精度之间自动转换
# 在启用AMP时通常使用float16来尝试加速计算并减少内存占用
ctx = nullcontext() if device_type == 'cpu' else torch.amp.autocast(device_type=device_type, dtype=ptdtype)
# 配置优化器
optimizer = model.configure_optimizers(weight_decay, learning_rate, (beta1, beta2), device_type)
checkpoint = None # 释放内存

# 估计损失
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            with ctx: #在GPU上自动使用float16计算
                _, loss = model(X, Y) # GPT类的forward方法第二个返回值即loss
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# 学习率衰减调度器(带预热的余弦算法)
def get_lr(it):
    # 1) 学习率从0开始线性增加到初始学习率，防止模型参数更新过快，确保模型的稳定性
    if it < warmup_iters:
        return learning_rate * it / warmup_iters
    # 2) 如果it > lr_decay_iters, 返回最小学习率
    if it > lr_decay_iters:
        return min_lr
    # 3) 当前迭代次数在预热和最大衰减迭代之间，函数将使用余弦衰减公式来计算学习率。
    decay_ratio = (it - warmup_iters) / (lr_decay_iters - warmup_iters)
    assert 0 <= decay_ratio <= 1
    # 实现周期性变化，即开始减小然后增大，在下一个周期再次减小
    coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio)) # coeff ranges 0..1
    return min_lr + coeff * (learning_rate - min_lr)

num decayed parameter tensors: 26, with 10,740,096 parameters
num non-decayed parameter tensors: 13, with 4,992 parameters
using fused AdamW: True


In [14]:
# 训练循环
X, Y = get_batch('train') # 获取第一个批量
t0 = time.time()
local_iter_num = 0 # 本进程生命周期中的迭代次数
decay_lr = True # 是需要设置学习衰减
running_mfu = -1.0
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
eval_only = False # 如果为True，则训练脚本在第一次eval之后退出
grad_clip = 1.0 # 使用参数裁剪梯度

In [15]:
while True:
    # 确定和设置本次迭代的学习率
    lr = get_lr(iter_num) if decay_lr else learning_rate
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

    # 评估训练或验证的损失，然后写进checkpoints里
    if iter_num % eval_interval == 0 :
        losses = estimate_loss()
        print(f"step {iter_num}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
        if losses['val'] < best_val_loss or always_save_checkpoint:
            best_val_loss = losses['val']
            if iter_num > 0:
                checkpoint = {
                    'model': model.state_dict(),
                    'optimizer': optimizer.state_dict(),
                    'model_args': model_args,
                    'iter_num': iter_num,
                    'best_val_loss': best_val_loss,
                    'config': config_str,
                }
                print(f"saving checkpoint to {out_dir}")
                torch.save(checkpoint, os.path.join(out_dir, 'ckpt.pt'))
    if iter_num == 0 and eval_only:
        break
        
    #向前向后更新，如果数据类型为float16，可选择梯度累积来模拟更大的批处理大小并使用GradScaler
    for micro_step in range(gradient_accumulation_steps):
        with ctx:
            logits, loss = model(X, Y)
            loss = loss / gradient_accumulation_steps # 按比例计算损失，以说明梯度积累
        # 当模型在GPU上进行前向传递时，立即异步预取下一批
        X, Y = get_batch('train')
        # 在fp16中进行梯度缩放训练
        scaler.scale(loss).backward()
    # 裁剪梯度
    if grad_clip != 0.0:
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
    # 在fp16中步进优化器和缩放器的训练
    scaler.step(optimizer)
    scaler.update()
    # 刷新梯度，然后释放内存
    optimizer.zero_grad(set_to_none=True)

    # 计时与日志
    t1 = time.time()
    dt = t1 - t0
    t0 = t1
    if iter_num % log_interval == 0:        
        lossf = loss.item() * gradient_accumulation_steps
        if local_iter_num >= 5: # 让训练稳定下来
            mfu = model.estimate_mfu(batch_size * gradient_accumulation_steps, dt)
            running_mfu = mfu if running_mfu == -1.0 else 0.9*running_mfu + 0.1*mfu
        print(f"iter {iter_num}: loss {lossf:.4f}, time {dt*1000:.2f}ms, mfu {running_mfu*100:.2f}%")
    iter_num += 1
    local_iter_num += 1

    # 终止条件
    if iter_num > max_iters:
        break

step 0: train loss 4.3926, val loss 4.3856
iter 0: loss 4.3597, time 317620.96ms, mfu -100.00%
iter 10: loss 3.2321, time 75.37ms, mfu 4.94%
iter 20: loss 2.8081, time 34.00ms, mfu 5.55%
iter 30: loss 2.6389, time 41.00ms, mfu 5.90%
iter 40: loss 2.5645, time 36.00ms, mfu 6.34%
step 50: train loss 2.5136, val loss 2.5185
saving checkpoint to ../data/gpt2
iter 50: loss 2.5301, time 352987.16ms, mfu 5.71%
iter 60: loss 2.5249, time 36.99ms, mfu 6.15%
iter 70: loss 2.4842, time 42.00ms, mfu 6.42%
iter 80: loss 2.4915, time 61.70ms, mfu 6.38%
iter 90: loss 2.4608, time 29.97ms, mfu 6.99%
step 100: train loss 2.4474, val loss 2.4782
saving checkpoint to ../data/gpt2
iter 100: loss 2.4637, time 353625.14ms, mfu 6.29%
iter 110: loss 2.4395, time 36.03ms, mfu 6.69%
iter 120: loss 2.4470, time 71.00ms, mfu 6.55%
iter 130: loss 2.4428, time 42.00ms, mfu 6.78%
iter 140: loss 2.4260, time 33.98ms, mfu 7.20%
step 150: train loss 2.3625, val loss 2.3981
saving checkpoint to ../data/gpt2
iter 150: lo

### 15.8.4.4 文本生成


In [11]:
"""
从以训练好模型中采样和生成文本
"""
import os
import pickle
from contextlib import nullcontext
import torch
import tiktoken
#from model import GPTConfig, GPT
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
# -----------------------------------------------------------------------------
# init_from = 'resume' # 'resume' 或者gpt2变体 (比如'gpt2-xl')
out_dir = '../data/gpt2' # 如果init_from参数不是 'resume'，则忽略该参数
start = "\n" # 或者为 "<|endoftext|>" ，也可以指定为一个文件比如: "FILE:prompt.txt"
num_samples = 10 # 生成的样本数
max_new_tokens = 500 # 每个样本生成的词元数
temperature = 0.8 # 在预测时，1.0 = 没变化, < 1.0 = 更小随机, > 1.0 = 更随机
top_k = 200 # 只保留top_k个可能性最大的词元，将其他词元的概率设为0
seed = 1337
device = 'cuda' # 比如: 'cpu', 'cuda', 'cuda:0', 'cuda:1'
dtype = 'bfloat16' if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else 'float16' # 'float32' 或者 'bfloat16' 或者 'float16'
compile = False # 使用Pytorch2.0以上版本支持compile功能，但是即便版本达到标准，在windows下也不是很可靠
#exec(open('configurator.py').read()) # 使用命令行或配置文件中的参数覆盖默认参数
# -----------------------------------------------------------------------------

In [12]:
torch.manual_seed(seed)# 设置种子可以确保每次运行代码时都能产生相同的随机数序列
torch.cuda.manual_seed(seed) #保证在CUDA上的操作有相同的随机性，尤其在使用GPU进行计算时
torch.backends.cuda.matmul.allow_tf32 = True # 它允许使用TensorFloat-32进行矩阵乘法matmul
torch.backends.cudnn.allow_tf32 = True # allow tf32 on cudnn
device_type = 'cuda' 
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
ctx = nullcontext() if device_type == 'cpu' else torch.amp.autocast(device_type=device_type, dtype=ptdtype)

In [13]:
# model
# 从前面训练好的模型中加载模型参数文件
ckpt_path = os.path.join(out_dir, 'ckpt.pt')
checkpoint = torch.load(ckpt_path, map_location=device)
gptconf = GPTConfig(**checkpoint['model_args'])
model = GPT(gptconf)
state_dict = checkpoint['model']
unwanted_prefix = '_orig_mod.'
for k,v in list(state_dict.items()):
    if k.startswith(unwanted_prefix):
        state_dict[k[len(unwanted_prefix):]] = state_dict.pop(k)
model.load_state_dict(state_dict)

number of parameters: 10.65M


In [14]:
model.eval()
model.to(device)

start_ids = encode(start)
x = (torch.tensor(start_ids, dtype=torch.long, device=device)[None, ...])

# run generation
with torch.no_grad():
    with ctx:
        for k in range(num_samples):
            y = model.generate(x, max_new_tokens, temperature=temperature, top_k=top_k)
            print(decode(y[0].tolist()))
            print('---------------')



And thy bridce.

STANUS:
The madere my be to take rudter the hands:
Whith foul heart. Whe dilth a endway, what fast to morous
You some for the coul the will at miree,
In con latist in overs, and the now they juse, less dience.

KING RICHARD:
Sup Maiss hiwhy hell and not of this death:
Is wor moth keard in on her evicks the moses
That in him he pood of his but thand thrup this ar is his shat thy ale of their Prience of my thy suk!

QUEEN ELIZABETH:
That he may in courrear tey I what I
And Edve y
---------------

Men pand king my thou cont beent of a cemose.

DUKE OF YORK:
Artand the caucil son the grove, with courfe:
I shout might to Enks fortul work.

LUCIO:
And thou to didings, to dess know thy see:
He dous the gRiet, I you, may sheask not my sided.

DUKE VINCENTIO:
Hatake begue here cowar som thy live on made so lack.

Prive, I my have, his I stard poiven's ond Camme,
Thou hat is is ofter not the summe of my and is be thand the some.

BRTHUS:
Thou so have reave is wat do it of the 

- **参考资料**
  - [Improving Language Understandingby Generative Pre-Training](https://cdn.openai.com/research-covers/language-unsupervised/language_understanding_paper.pdf)
  - [OpenAI ChatGPT（二）：十分钟读懂 GPT-1](https://zhuanlan.zhihu.com/p/604625917)
  - [ChatGPT/InstructGPT详解](https://zhuanlan.zhihu.com/p/590311003)
  - [GPT-4核心技术探秘](https://zhuanlan.zhihu.com/p/626463196)
  - [图解GPT-2 | The Illustrated GPT-2](https://lolitasian.blog.csdn.net/article/details/125529598)(强烈推荐)